# BGS Signal-to-Noise Ratio and Redshift Efficiency

The goal of this notebook is to assess the signal-to-noise ratio and redshift efficiency of BGS targets observed in "nominal" observing conditions (which are defined [here](https://github.com/desihub/desisurvey/blob/master/py/desisurvey/data/config.yaml#L102) and discussed [here](https://github.com/desihub/desisurvey/issues/77), among other places).  Specifically, the nominal BGS observing conditions we adopt (note the 5-minute exposure time is with the moon down!) are:

```python
{'AIRMASS': 1.0,
 'EXPTIME': 300,
 'SEEING': 1.1,
 'MOONALT': -60,
 'MOONFRAC': 0.0,
 'MOONSEP': 180}
```

During the survey itself, observations with the moon up (i.e., during bright time) will be obtained with longer exposure times according to the bright-time exposure-time model (see [here](https://github.com/desihub/surveysim/tree/master/doc/nb)).

Because we fix the observing conditions, we only consider how redshift efficiency depends on galaxy properties (apparent magnitude, redshift, 4000-A break, etc.).  However, note that the code is structured such that we *could* (now or in the future) explore variations in seeing, exposure time, and lunar parameters.

For code to generate large numbers of spectra over significant patches of sky and to create a representative DESI dataset (with parallelism), see `desitarget/bin/select_mock_targets` and `desitarget.mock.build.targets_truth`.

Finally, note that the various python Classes instantiated here (documented in `desitarget.mock.mockmaker`) are easily extensible to other mock catalogs and galaxy/QSO/stellar physics.



In [1]:
import os
import numpy as np
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
from astropy.table import Table, vstack
from astropy.io import fits

/global/common/software/desi/cori/desiconda/20180130-1.2.4-spec/conda/lib/python3.6/site-packages/ipykernel/__main__.py:4: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/global/common/software/desi/cori/desiconda/20180130-1.2.4-spec/conda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/global/common/software/desi/cori/desiconda/20180130-1.2.4-spec/conda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/global/common/software/desi/cori/desiconda/20180130-1.2.4-spec/conda/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/global/common/software/desi/cori/desiconda/201

In [2]:
from desispec.io.util import write_bintable
from desiutil.log import get_logger, DEBUG
log = get_logger()
from desitarget.cuts import isBGS_bright, isBGS_faint
## Following not yet available in the master branch
from desitarget.mock.mockmaker import BGSMaker
from desitarget.mock.mockmaker import SKYMaker

In [3]:
import multiprocessing
nproc = multiprocessing.cpu_count() // 2 

import seaborn as sns
sns.set(style='white', font_scale=1.1, palette='deep')

In [4]:
# Specify if using this from command line as a .py or as an ipynb
using_py = False

class arg:
    pass

In [5]:
simnames = ['sim46']#['sim13','sim14','sim16','sim17','sim18'] #'sim12',

In [6]:
if using_py:
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument('--sim', type=int, default=None, help='Simulation number (see documentation)')
    parser.add_argument('--part', type=str, default=None, help='Which part of the simulation to run. Options are all, newexp, group, zfit')
    args = parser.parse_args()
    if args.sim is None:
        parser.print_help()
        sys.exit(1)   
else:
    %matplotlib inline
    %load_ext autoreload
    %autoreload 2
    args = arg()
    args.sim = 1
    args.part = 'all'

#### Establish the I/O path, random seed, and path to the dust maps and desired healpixel.

In [7]:
simdir = os.path.join(os.getenv('DESI_ROOT'), 'spectro', 'sim', 'bgs', 'kremin', 'flat_priors')

if not os.path.exists(simdir):
    os.makedirs(simdir)

In [8]:
seed = 626

### All or none of the output files can be overwritten using these keywords.

In [9]:
overwrite_spectra = True
#overwrite_templates = overwrite_spectra
overwrite_redshifts = True
overwrite_results = True

#### Initialize random state

In [10]:
rand = np.random.RandomState(seed)

## Set up the simulation parameters.

Here we use the mock to capture the correct distribution of apparent magnitudes, galaxy properties, and redshifts.

Note that if `use_mock=False` then *rmagmin*, *rmagmax*, *zmin*, and *zmax* are required. For example, here's another possible simulation of 1000 spectra in which the magnitude (r=19.5) and redshift (z=0.2) are held fixed while moonfrac and moonsep are varied (as well as intrinsic galaxy properties):

```python
sim2 = dict(suffix='sim02',
            use_mock=False,
            nsim=10,
            nspec=100,
            seed=22,
            zmin=0.2, zmax=0.2,
            rmagmin=19.5, rmagmax=19.5,
            moonfracmin=0.0, moonfracmax=1.0,
            moonsepmin=0.0, moonsepmax=120.0,
           )
```

In [11]:
from desistudy import get_predefined_sim_dict, get_predefined_obs_dict

all_sims = []
all_obsconds = []

for simname in simnames:
    all_sims.append(get_predefined_sim_dict(simname))
    all_obsconds.append(get_predefined_obs_dict(simname))

print(all_obsconds)

sims = np.atleast_1d(all_sims)
conditions = np.atleast_1d(all_obsconds)

[{'AIRMASS': 1.0, 'SEEING': 1.1, 'MOONALT': 90, 'MOONSEP': 20, 'EXPTIME': 300, 'MOONFRAC': 0.99}]


### Generate Spectra

In [16]:
from desistudy import bgs_sim_spectra

if overwrite_spectra:
    for sim,cond in zip(sims,conditions):
        log.info("\n\n\n\nNow performing sim {}".format(sim['suffix']))
        bgs_sim_spectra(sim, cond, simdir, verbose=False, overwrite=overwrite_spectra)
        log.info("\n\nFinished simulating templates\n\n")  

INFO:<ipython-input-16-068526d5d605>:5:<module>: 



Now performing sim sim46
INFO:io.py:1013:read_basis_templates: Reading /global/project/projectdirs/desi/spectro/templates/basis_templates/v2.5/bgs_templates_v2.1.fits metadata.
INFO:io.py:1025:read_basis_templates: Reading /global/project/projectdirs/desi/spectro/templates/basis_templates/v2.5/bgs_templates_v2.1.fits
Writing /global/project/projectdirs/desi/spectro/sim/bgs/kremin/flat_priors/sim46/bgs-sim46-simdata.fits
{'AIRMASS': 1.0, 'EXPTIME': 300.0, 'MOONALT': 90.0, 'MOONFRAC': 0.99000001, 'MOONSEP': 20.0, 'SEEING': 1.1}


Exception: 

## Fit the redshifts.

This step took ~1.8 seconds per spectrum, ~3 minutes per 100 spectra, or ~30 minutes for all 1000 spectra with my 4-core laptop.

In [13]:
from desistudy import bgs_redshifts

if overwrite_redshifts:
    for sim in sims:
        log.info("\n\n\n\nNow performing sim {}".format(sim['suffix']))
        bgs_redshifts(sim, simdir=simdir, overwrite=overwrite_redshifts)
        log.info("\n\n\n\n\nFinished redshift fitting\n\n\n")

INFO:<ipython-input-13-3473eae6755c>:5:<module>: 



Now performing sim sim46
Running on a NERSC login node- reducing number of processes to 4
Running with 4 processes
Loading targets...
Read and distribution of 800 targets: 14.4 seconds
DEBUG: Using default redshift range 0.0050-1.6988 for rrtemplate-galaxy.fits
DEBUG: Using default redshift range 0.5000-3.9956 for rrtemplate-qso.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-A.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-B.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-Carbon.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-F.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-G.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-K.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-Ldwarf.fits
DEBUG: Using default redshift range -0.0020-0.0020 for 

  Scanning redshifts for template STAR:::G
    Progress:   0 %
    Progress:  10 %
    Progress:  20 %
    Progress:  30 %
    Progress:  40 %
    Progress:  50 %
    Progress:  60 %
    Progress:  70 %
    Progress:  80 %
    Progress:  90 %
    Progress: 100 %
    Finished in: 15.5 seconds
  Scanning redshifts for template STAR:::K
    Progress:   0 %
    Progress:  10 %
    Progress:  20 %
    Progress:  30 %
    Progress:  40 %
    Progress:  50 %
    Progress:  60 %
    Progress:  70 %
    Progress:  80 %
    Progress:  90 %
    Progress: 100 %
    Finished in: 15.5 seconds
  Scanning redshifts for template STAR:::LDWARF
    Progress:   0 %
    Progress:  10 %
    Progress:  20 %
    Progress:  30 %
    Progress:  40 %
    Progress:  50 %
    Progress:  60 %
    Progress:  70 %
    Progress:  80 %
    Progress:  90 %
    Progress: 100 %
    Finished in: 4.9 seconds
  Scanning redshifts for template STAR:::M
    Progress:   0 %
    Progress:  10 %
    Progress:  20 %
    Progress: 

## Gather the results.

In [14]:
from desistudy import bgs_gather_results

if overwrite_results:
    for sim in sims:
        log.info("\n\n\n\nNow performing sim {}".format(sim['suffix']))
        bgs_gather_results(sim, simdir=simdir, overwrite=overwrite_results)
        log.info("Finished gathering results")

INFO:<ipython-input-14-299839a56a8e>:5:<module>: 



Now performing sim sim46
INFO:desistudy.py:209:bgs_gather_results: Reading /global/project/projectdirs/desi/spectro/sim/bgs/kremin/flat_priors/sim46/bgs-sim46-000-true.fits
INFO:desistudy.py:223:bgs_gather_results: Reading /global/project/projectdirs/desi/spectro/sim/bgs/kremin/flat_priors/sim46/bgs-sim46-000-zbest.fits
INFO:desistudy.py:235:bgs_gather_results: Reading /global/project/projectdirs/desi/spectro/sim/bgs/kremin/flat_priors/sim46/bgs-sim46-000.fits


INFO:desistudy.py:209:bgs_gather_results: Reading /global/project/projectdirs/desi/spectro/sim/bgs/kremin/flat_priors/sim46/bgs-sim46-001-true.fits
INFO:desistudy.py:223:bgs_gather_results: Reading /global/project/projectdirs/desi/spectro/sim/bgs/kremin/flat_priors/sim46/bgs-sim46-001-zbest.fits
INFO:desistudy.py:235:bgs_gather_results: Reading /global/project/projectdirs/desi/spectro/sim/bgs/kremin/flat_priors/sim46/bgs-sim46-001.fits
INFO:desistudy.py:246:bgs_gather_results: Writing /global/project/projectdirs/desi/spectro/sim/bgs/kremin/flat_priors/sim46/bgs-sim46-results.fits
INFO:<ipython-input-14-299839a56a8e>:7:<module>: Finished gathering results


# Do everything in one cell

In [15]:
# from desistudy import bgs_sim_spectra
# from desistudy import bgs_redshifts
# from desistudy import bgs_gather_results

# for sim,cond in zip(sims,conditions):
#     log.info("\n\n\n\nNow performing sim {}".format(sim['suffix']))
#     if overwrite_spectra:
#         bgs_sim_spectra(sim, cond, simdir, verbose=False, overwrite=overwrite_spectra)
#         log.info("Finished simulating templates")
#     if overwrite_redshifts:
#         bgs_redshifts(sim, simdir=simdir, overwrite=overwrite_redshifts)
#         log.info("Finished redshift fitting")
#     if overwrite_results:
#         bgs_gather_results(sim, simdir=simdir, overwrite=overwrite_results)
#         log.info("Finished gathering results")